# Batch Segmentation for Remote Sensing Imagery with SAM 3

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/sam3_batch_segmentation.ipynb)

This notebook demonstrates how to do batch segmentation for remote sensing imagery with SAM 3.

## Installation

First, make sure you have the required dependencies installed:

In [2]:
%pip install "segment-geospatial[samgeo3]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.9/632.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 131.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 118.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 133.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 69.6 MB/s eta 0:00:00
   ━━━━━

## Import Libraries


In [3]:
import leafmap
from samgeo import SamGeo3, download_file

To use SamGeo 2, install it as:
	pip install segment-geospatial[samgeo2]


## Download Sample Data

Let's download sample satellite images covering the University of California, Berkeley, for testing:


In [4]:
image_paths = []
for i in range(1, 5):
    url = f"https://huggingface.co/datasets/giswqs/geospatial/resolve/main/uc_berkeley_{i}.tif"
    image_path = download_file(url)
    image_paths.append(image_path)

Downloading...
From: https://huggingface.co/datasets/giswqs/geospatial/resolve/main/uc_berkeley_1.tif
To: /content/uc_berkeley_1.tif
100%|██████████| 142/142 [00:00<00:00, 156kB/s]
Downloading...
From: https://huggingface.co/datasets/giswqs/geospatial/resolve/main/uc_berkeley_2.tif
To: /content/uc_berkeley_2.tif
100%|██████████| 142/142 [00:00<00:00, 420kB/s]
Downloading...
From: https://huggingface.co/datasets/giswqs/geospatial/resolve/main/uc_berkeley_3.tif
To: /content/uc_berkeley_3.tif
100%|██████████| 142/142 [00:00<00:00, 419kB/s]
Downloading...
From: https://huggingface.co/datasets/giswqs/geospatial/resolve/main/uc_berkeley_4.tif
To: /content/uc_berkeley_4.tif
100%|██████████| 9.35M/9.35M [00:00<00:00, 75.2MB/s]


In [10]:
from PIL import Image
img = Image.open("/content/uc_berkeley_1.tif")
img.show()

UnidentifiedImageError: cannot identify image file '/content/uc_berkeley_1.tif'

In [11]:
!file /content/uc_berkeley_1.tif

/content/uc_berkeley_1.tif: HTML document, ASCII text, with no line terminators


In [12]:
with open("/content/uc_berkeley_1.tif", "r") as f:
    print(f.read()[:500])

<!doctype html><meta charset="utf-8"><meta name=viewport content="width=device-width, initial-scale=1"><title>429</title>429 Too Many Requests


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import os
print(os.path.exists("/content/drive/MyDrive/uc_berkeley_1.tif"))

True


In [21]:
image_paths = [
    "/content/drive/MyDrive/uc_berkeley_1.tif",
    "/content/drive/MyDrive/uc_berkeley_2.tif",
    "/content/drive/MyDrive/uc_berkeley_3.tif",
    "/content/drive/MyDrive/uc_berkeley_4.tif"
]

In [19]:
m = leafmap.Map()
for i in range(len(image_paths)):
    m.add_raster(image_paths[i], layer_name=f"image_{i + 1}")
m



Map(center=[37.8715, -122.2635], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

## Request access to SAM3

To use SAM3, you need to request access by filling out this form on Hugging Face: https://huggingface.co/facebook/sam3

Once you have access, uncomment the following code block and run it.

In [6]:
from huggingface_hub import login
login()

## Initialize SAM3

When initializing SAM3, you can choose the backend from "meta", or "transformers".

In [20]:
sam3 = SamGeo3(backend="meta", device=None, checkpoint_path=None, load_from_HF=True)

Using cuda device and meta backend


## Set the image batch

In [22]:
sam3.set_image_batch(image_paths)

Set 4 images for batch processing.


## Generate masks with text prompt

Generate masks for all images with a text prompt

In [23]:
sam3.generate_masks_batch("building", min_size=100)

Processed 4 image(s), found 175 total object(s).


In [24]:
# Access results for each image
for i, result in enumerate(sam3.batch_results):
    print(f"Image {i + 1}: Found {len(result['masks'])} objects")
    # result contains: masks, boxes, scores, image, source

Image 1: Found 47 objects
Image 2: Found 50 objects
Image 3: Found 64 objects
Image 4: Found 14 objects


## Show results

In [1]:
# Visualize all annotations in a grid
sam3.show_anns_batch(ncols=2, show_bbox=True, show_score=True)

NameError: name 'sam3' is not defined

![](https://github.com/user-attachments/assets/0c4f0b4b-4104-4a43-9173-ab76e0d2d71a)

## Save results

In [ ]:
sam3.show_anns_batch(output_dir="output/annotations/", prefix="ann", dpi=300)

In [ ]:
# Save all masks to disk
saved_files = sam3.save_masks_batch(
    output_dir="output/", prefix="building_mask", unique=True
)